In [4]:
import pandas as pd

In [5]:
dates = pd.read_gbq(
    query='select distinct date(timestamp) as date from solana.deposits',
    project_id='friktion-dev'
    )

In [6]:
dates.describe()

,date
count,110
unique,110
top,2022-04-04
freq,1


In [7]:
users = pd.read_gbq(
    query='select distinct(userAddress) as userAddress from solana.deposits')

In [8]:
users.describe()

,userAddress
count,13893
unique,13893
top,wGm3yCrVppHYratwKAv9pBNRbge18rpn9XiAujZ6jwF
freq,1


In [9]:
user_first_deposits_dates = pd.read_gbq(
    query='select userAddress, min(date(timestamp)) as first_deposit_dt from solana.deposits group by userAddress')

In [10]:
user_first_deposits_dates.describe()

,userAddress,first_deposit_dt
count,13893,13893
unique,13893,110
top,wGm3yCrVppHYratwKAv9pBNRbge18rpn9XiAujZ6jwF,2022-03-30
freq,1,2158


In [11]:
users.iloc[0][0]

'wGm3yCrVppHYratwKAv9pBNRbge18rpn9XiAujZ6jwF'

In [12]:
user_first_deposits_dates[user_first_deposits_dates['userAddress'].str.contains(users.iloc[0][0])]

,userAddress,first_deposit_dt
0,wGm3yCrVppHYratwKAv9pBNRbge18rpn9XiAujZ6jwF,2022-04-04


In [13]:
# eventuall you'll iterate through this activity of making a user table per userAddress and then unioning together with all the others

In [40]:
# to make this iterate through all users increment the first value in the iloc call
current_user_address = users.iloc[3][0]
user_first_deposit_date = user_first_deposits_dates[user_first_deposits_dates['userAddress'].str.contains(current_user_address)]['first_deposit_dt']

In [41]:
user_deposit_table = pd.DataFrame(columns=['userAddress', 'asOfDate', 'totalValueLockedUSD', 'totalDepositedUSD', 'totalWithdrawnUSD', \
                                           'tvlDelta30Days', 'tvlDelta60Days', 'tvlDelta90Days', \
                                           'tvlDelta1Epoch', 'tvlDelta2Epoch', \
                                           'tvlDelta3Epoch', 'tvlDelta4Epoch', 'tvlDelta5Epoch', \
                                           'firstDepositDate', 'firstDepositEpoch', 'firstDepositAmount', 'firstDepositToken', \
                                           'lastDepositDate', 'lastDepositEpoch', 'lastDepositAmount', 'lastDepositToken', \
                                           'daysSinceLastDeposit', 'epochsSinceLastDeposit', \
                                           'firstWithdrawalDate', 'firstWithdrawalEpoch', 'firstWithdrawalAmount', 'firstWithdrawalToken', \
                                           'lastWithdrawalDate', 'lastWithdrawalEpoch', 'lastWithdrawalAmount', 'lastWithdrawalToken', \
                                           'hasChurned', 'churnDate', 'churnEpoch'])

### asOfDate

In [42]:
user_deposit_table['asOfDate'] = dates[dates['date'].apply(lambda x: x>=user_first_deposit_date).iloc[:,0]]

### userAddress

In [43]:
user_deposit_table['userAddress'] = current_user_address

### totalValueDepositedUSD
TODO: Add logic to covert to USD

#### inner loop logic

In [44]:
query_date = user_deposit_table['asOfDate'][0].isoformat()
pd.read_gbq(query=f"select sum(amount) as total_deposited from solana.deposits where userAddress=\'{current_user_address}\' and date(timestamp) <= date(\'{query_date}\')")

,total_deposited
0,2.479377


In [45]:
# need to add market marking code
user_deposit_table['totalDepositedUSD'] = [pd.read_gbq(query=f"select sum(amount) as total_deposited from solana.deposits where userAddress=\'{current_user_address}\' and date(timestamp) <= date(\'{query_date.isoformat()}\')").iloc[0,0] for query_date in user_deposit_table['asOfDate'].tolist()]

### totalValueWithdrawnUSD
TODO: Add logic to convert to USD

In [46]:
query_date = user_deposit_table['asOfDate'][0].isoformat()
pd.read_gbq(query=f"select sum(amount) as total_withdrawn from solana.withdrawals where userAddress=\'{current_user_address}\' and date(timestamp) <= date(\'{query_date}\')").fillna(0).iloc[0,0]

0.380040385

In [47]:
# need to add market marking code
user_deposit_table['totalWithdrawnUSD'] = [pd.read_gbq(query=f"select sum(amount) as total_withdrawn from solana.withdrawals where userAddress=\'{current_user_address}\' and date(timestamp) <= date(\'{query_date.isoformat()}\')").fillna(0).iloc[0,0] for query_date in user_deposit_table['asOfDate'].tolist()]

### totalValueLockedUSD

In [48]:
user_deposit_table['totalValueLockedUSD'] = user_deposit_table['totalDepositedUSD'] - user_deposit_table['totalWithdrawnUSD']

### firstDepositDate

In [49]:
first_deposit_date = user_first_deposits_dates[user_first_deposits_dates['userAddress'].str.match(current_user_address)]['first_deposit_dt'].iloc[0]
user_deposit_table['firstDepositDate'] =  first_deposit_date

### firstDepositEpoch

### firstDepositAmount

In [50]:
user_deposit_table['firstDepositAmount'] = pd.read_gbq(query=f'select sum(amount) from solana.deposits where userAddress = \'{current_user_address}\' and date(timestamp) = \'{first_deposit_date.isoformat()}\'').iloc[0,0]

### firstDepositToken

In [51]:
user_deposit_table['firstDepositToken'] = pd.read_gbq(query=f'select currencyName from solana.deposits where userAddress = \'{current_user_address}\' and date(timestamp) = \'{first_deposit_date.isoformat()}\'').iloc[0,0]

### tvlDelta30Days

In [63]:
current_user_address

'ANE7Nsw5zuF4sdrkGr1Q4ETHYXZu7wyQ4Tq8hhaa9mxE'

In [70]:
import datetime

current_as_of_date = user_deposit_table['asOfDate'].iloc[3]

# we can also perform this logic with GBQ SQL
#      DATE_ADD(DATE "2022-01-19", INTERVAL 30 DAY)
window_end_date = current_as_of_date + datetime.timedelta(30)

pd.read_gbq(query=f'\
    with window_deposits as \
         (select userAddress, sum(amount) as deposits \
         from solana.deposits \
         where userAddress = \'{current_user_address}\' \
         and date(timestamp) between \'{current_as_of_date.isoformat()}\' and \'{window_end_date}\'\
         group by userAddress) \
    , window_withdrawals as \
        (select userAddress, sum(amount) as withdrawals \
        from solana.withdrawals \
        where userAddress = \'{current_user_address}\' \
        and date(timestamp) between \'{current_as_of_date.isoformat()}\' and \'{window_end_date}\'\
        group by userAddress) \
    select \
         (deposits - withdrawals) \
    from window_deposits wd\
    left join window_withdrawals ww on wd.userAddress = ww.userAddress \
    ')

#there is an interesting loss of precision when going from the GBQ to pandas GBQ

,f0_


In [68]:
tvl_list = []
for table_date in user_deposit_table['asOfDate']:
    print(table_date)
    current_as_of_date = table_date
    window_end_date = current_as_of_date + datetime.timedelta(30)
    window_delta = pd.read_gbq(query=f'\
        with window_deposits as \
             (select userAddress, sum(amount) as deposits \
             from solana.deposits \
             where userAddress = \'{current_user_address}\' \
             and date(timestamp) between \'{current_as_of_date.isoformat()}\' and \'{window_end_date}\'\
             group by userAddress) \
        , window_withdrawals as \
            (select userAddress, sum(amount) as withdrawals \
            from solana.withdrawals \
            where userAddress = \'{current_user_address}\' \
            and date(timestamp) between \'{current_as_of_date.isoformat()}\' and \'{window_end_date}\'\
            group by userAddress) \
        select \
             (deposits - withdrawals) \
        from window_deposits wd\
        left join window_withdrawals ww on wd.userAddress = ww.userAddress \
        ').iloc[0,0]
    tvl_list.append(window_delta)
    
user_deposit_table['tvlDelta30Days'] = tvl_list

2022-04-04
2022-04-03
2022-03-11
2022-01-30


IndexError: single positional indexer is out-of-bounds

### tvlDelta60Days

### tvlDelta90Days

In [55]:
user_deposit_table.sort_values('asOfDate')

,userAddress,asOfDate,totalValueLockedUSD,totalDepositedUSD,totalWithdrawnUSD,tvlDelta30Days,tvlDelta60Days,tvlDelta90Days,tvlDelta1Epoch,tvlDelta2Epoch,...,firstWithdrawalEpoch,firstWithdrawalAmount,firstWithdrawalToken,lastWithdrawalDate,lastWithdrawalEpoch,lastWithdrawalAmount,lastWithdrawalToken,hasChurned,churnDate,churnEpoch
23,ANE7Nsw5zuF4sdrkGr1Q4ETHYXZu7wyQ4Tq8hhaa9mxE,2022-01-19,0.400000,0.400000,0.00000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,ANE7Nsw5zuF4sdrkGr1Q4ETHYXZu7wyQ4Tq8hhaa9mxE,2022-01-20,0.400000,0.400000,0.00000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43,ANE7Nsw5zuF4sdrkGr1Q4ETHYXZu7wyQ4Tq8hhaa9mxE,2022-01-21,0.400000,0.400000,0.00000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87,ANE7Nsw5zuF4sdrkGr1Q4ETHYXZu7wyQ4Tq8hhaa9mxE,2022-01-22,0.400000,0.400000,0.00000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,ANE7Nsw5zuF4sdrkGr1Q4ETHYXZu7wyQ4Tq8hhaa9mxE,2022-01-23,0.400000,0.400000,0.00000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37,ANE7Nsw5zuF4sdrkGr1Q4ETHYXZu7wyQ4Tq8hhaa9mxE,2022-03-31,1.868908,2.248948,0.38004,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,ANE7Nsw5zuF4sdrkGr1Q4ETHYXZu7wyQ4Tq8hhaa9mxE,2022-04-01,1.868908,2.248948,0.38004,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,ANE7Nsw5zuF4sdrkGr1Q4ETHYXZu7wyQ4Tq8hhaa9mxE,2022-04-02,1.868908,2.248948,0.38004,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ANE7Nsw5zuF4sdrkGr1Q4ETHYXZu7wyQ4Tq8hhaa9mxE,2022-04-03,1.868908,2.248948,0.38004,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
user_deposit_table.melt()

,variable,value
0,userAddress,ANE7Nsw5zuF4sdrkGr1Q4ETHYXZu7wyQ4Tq8hhaa9mxE
1,userAddress,ANE7Nsw5zuF4sdrkGr1Q4ETHYXZu7wyQ4Tq8hhaa9mxE
2,userAddress,ANE7Nsw5zuF4sdrkGr1Q4ETHYXZu7wyQ4Tq8hhaa9mxE
3,userAddress,ANE7Nsw5zuF4sdrkGr1Q4ETHYXZu7wyQ4Tq8hhaa9mxE
4,userAddress,ANE7Nsw5zuF4sdrkGr1Q4ETHYXZu7wyQ4Tq8hhaa9mxE
...,...,...
2579,churnEpoch,NaN
2580,churnEpoch,NaN
2581,churnEpoch,NaN
2582,churnEpoch,NaN


### tvlDelta1Epoch

### tvlDelta2Epoch

### tvlDelta3Epoch

### tvlDelta4Epoch

### tvlDelta5Epoch

### lastDepositDate

### lastDepositEpoch

### lastDepositAmount

### lastDepositToken

### daysSinceLastDeposit

### epochsSinceLastDeposit

### firstWithdrawalDate

### firstWithdrawalEpoch

### firstWithdrawalAmount

### firstWithdrawalToken

### lastWithdrawalDate

### lastWithdrawalEpoch

### lastWithdrawalAmount

### lastWithdrawalToken

### hasChurned

### chrunDate

### churnEpoch